# Import Patient Data Tables

We will import all patient-related CSV files from the `/raw/readmission/` folder using pandas.

This cell imports pandas and loads multiple CSV files related to hospital admissions, lab events, diagnoses, procedures, and patient data. The datasets appear to be part of a healthcare database (possibly MIMIC or similar) and include:

- Admissions (Patient hospitalization records)

- CPT Events (Current Procedural Terminology, billing codes)

- Lab Items Dictionary (Metadata for lab tests)

- Diagnoses (ICD codes) (Patient diagnoses, likely ICD-9/10)

- DRG Codes (Diagnosis-Related Groups for billing)

- Lab Events (Actual lab test results)

- Patients (Demographic/static patient data)

- Procedures (ICD codes) (Medical procedures performed)

In [1]:
import pandas as pd

# Define file paths
data_path = '../raw/readmission/'
admissions = pd.read_csv(data_path + 'admissions_202208161605.csv')
cptevents = pd.read_csv(data_path + 'cptevents_202208161605.csv' , low_memory=False)
d_labitems = pd.read_csv(data_path + 'd_labitems_202208161605.csv')
diagnoses_icd = pd.read_csv(data_path + 'diagnoses_icd_202208161605.csv')
drgcodes = pd.read_csv(data_path + 'drgcodes_202208161605.csv')
labevents = pd.read_csv(data_path + 'labevents_202208161605.csv')
patients = pd.read_csv(data_path + 'patients_202208161605.csv')
procedures_icd = pd.read_csv(data_path + 'procedures_icd_202208161605.csv')


In [2]:
heart_codes = [
    '39891','40201','40211','40291','40401','40403','40411','40413',
    '40491','40493','4280','4281','42820','42821','42822','42823',
    '42830','42831','42832','42833','42840','42841','42842','42843','4289'
]

# Step 1: Filter heart failure diagnoses and merge with admissions
hf_diagnoses = diagnoses_icd[diagnoses_icd['icd9_code'].isin(heart_codes)]
hf_admissions = pd.merge(
    hf_diagnoses[['subject_id', 'hadm_id']],
    admissions[['hadm_id', 'subject_id', 'admittime', 'dischtime', 'deathtime', 
                'admission_type', 'discharge_location', 
                'marital_status', 'hospital_expire_flag']],
    on=['subject_id', 'hadm_id'],
    how='left'
).drop_duplicates(['subject_id', 'hadm_id'])

hf_admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag
0,115,114585,2194-10-16 12:48:00.000,2194-11-13 16:50:00.000,NaN,EMERGENCY,SNF,SINGLE,0
1,117,140784,2133-04-07 16:29:00.000,2133-04-12 16:39:00.000,NaN,EMERGENCY,HOME HEALTH CARE,SINGLE,0
2,124,138376,2165-12-31 18:55:00.000,2166-02-01 06:55:00.000,2166-02-01 06:55:00.000,EMERGENCY,DEAD/EXPIRED,MARRIED,1
4,130,198214,2119-10-29 14:49:00.000,2119-11-05 17:00:00.000,NaN,EMERGENCY,HOME HEALTH CARE,SINGLE,0
5,68,108329,2174-01-04 22:21:00.000,2174-01-19 11:30:00.000,NaN,EMERGENCY,SNF,SINGLE,0
...,...,...,...,...,...,...,...,...,...
21267,97132,144063,2180-11-23 16:38:00.000,2180-11-28 18:07:00.000,2180-11-28 18:07:00.000,EMERGENCY,DEAD/EXPIRED,WIDOWED,1
21268,97144,109999,2187-02-07 14:00:00.000,2187-02-27 13:15:00.000,NaN,ELECTIVE,REHAB/DISTINCT PART HOSP,WIDOWED,0
21270,97172,133092,2174-09-07 18:40:00.000,2174-09-14 19:58:00.000,NaN,EMERGENCY,HOSPICE-HOME,MARRIED,0
21272,97488,152542,2128-04-08 20:58:00.000,2128-04-13 16:27:00.000,NaN,EMERGENCY,HOME HEALTH CARE,DIVORCED,0


In [3]:
# Note: Convert date columns to datetime
hf_admissions['admittime'] = pd.to_datetime(hf_admissions['admittime'])
hf_admissions['dischtime'] = pd.to_datetime(hf_admissions['dischtime'])
hf_admissions['deathtime'] = pd.to_datetime(hf_admissions['deathtime'])

In [4]:
# Step 2: Define 30-day readmission target
hf_admissions_sorted = hf_admissions.sort_values(['subject_id', 'dischtime'])
hf_admissions_sorted['next_admittime'] = hf_admissions_sorted.groupby('subject_id')['admittime'].shift(-1)
hf_admissions_sorted['days_to_readmit'] = (
    (hf_admissions_sorted['next_admittime'] - hf_admissions_sorted['dischtime']).dt.total_seconds() / 86400
)

hf_admissions_sorted

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag,next_admittime,days_to_readmit
11,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,SNF,MARRIED,0,NaT,NaN
12,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,DEAD/EXPIRED,NaN,1,NaT,NaN
13,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,REHAB/DISTINCT PART HOSP,MARRIED,0,NaT,NaN
14,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,HOME,SINGLE,0,NaT,NaN
15,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaT,URGENT,HOME HEALTH CARE,MARRIED,0,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21244,99982,151454,2156-11-28 11:56:00,2156-12-08 13:45:00,NaT,EMERGENCY,HOME HEALTH CARE,MARRIED,0,2157-01-05 17:27:00,28.154167
5038,99982,112748,2157-01-05 17:27:00,2157-01-12 13:00:00,NaT,EMERGENCY,HOME,MARRIED,0,2157-02-16 17:31:00,35.188194
21182,99982,183791,2157-02-16 17:31:00,2157-02-22 20:36:00,NaT,EMERGENCY,SHORT TERM HOSPITAL,MARRIED,0,NaT,NaN
21183,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,ELECTIVE,HOME,MARRIED,0,NaT,NaN


In [5]:
# Target: 1 if readmitted in ≤30 days (exclude deaths/transfers)
hf_admissions_sorted['readmit_30'] = (
    (hf_admissions_sorted['days_to_readmit'] <= 30) & 
    (hf_admissions_sorted['days_to_readmit'] > 0) &
    (hf_admissions_sorted['deathtime'].isna()) &
    (~hf_admissions_sorted['discharge_location'].isin(['DIED', 'HOSPICE']))
).astype(int)

hf_admissions_sorted[hf_admissions_sorted['readmit_30'] == 1]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag,next_admittime,days_to_readmit,readmit_30
6,68,170467,2173-12-15 16:16:00,2174-01-03 18:30:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2174-01-04 22:21:00,1.160417,1
124,356,135591,2132-08-05 00:09:00,2132-08-13 17:41:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2132-08-31 17:05:00,17.975000,1
226,518,143946,2109-06-27 04:45:00,2109-06-29 17:00:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-07 07:49:00,7.617361,1
227,518,153168,2109-07-07 07:49:00,2109-07-09 12:16:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-17 04:21:00,7.670139,1
167,530,151499,2119-03-13 16:33:00,2119-04-04 12:00:00,NaT,EMERGENCY,REHAB/DISTINCT PART HOSP,MARRIED,0,2119-04-07 16:14:00,3.176389,1
...,...,...,...,...,...,...,...,...,...,...,...,...
20761,99439,143661,2179-02-22 01:26:00,2179-02-25 15:50:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-18 11:27:00,20.817361,1
20763,99439,145935,2179-03-18 11:27:00,2179-03-20 18:00:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-27 20:45:00,7.114583,1
20991,99538,194801,2180-02-13 17:46:00,2180-02-26 15:35:00,NaT,EMERGENCY,SNF,UNKNOWN (DEFAULT),0,2180-03-12 16:01:00,15.018056,1
21053,99562,129689,2183-02-02 15:13:00,2183-02-18 13:28:00,NaT,URGENT,SNF,MARRIED,0,2183-03-06 01:31:00,15.502083,1


In [6]:
import numpy as np

def calculate_age_robust(dob, admittime):
    """
    Calculate age handling all edge cases:
    - Missing values
    - Invalid dates
    - Date ranges where dob > admittime
    - Ages > 120 years
    - Month/day adjustments
    """
    # Handle missing values
    if pd.isna(dob) or pd.isna(admittime):
        return np.nan
    
    # Ensure datetime (if not already)
    dob = pd.to_datetime(dob, errors='coerce')
    admittime = pd.to_datetime(admittime, errors='coerce')
    
    # Check for conversion failures
    if pd.isna(dob) or pd.isna(admittime):
        return np.nan
    
    # Validate reasonable date ranges
    if (dob.year < 1900) or (dob > admittime):
        return np.nan
    
    # Calculate using date components (most reliable method)
    years = admittime.year - dob.year
    months = admittime.month - dob.month
    days = admittime.day - dob.day
    
    # Adjust if birthday hasn't occurred yet
    if (months < 0) or (months == 0 and days < 0):
        years -= 1
    
    # Return capped age (0-120)
    return max(0, min(years, 120))

In [7]:
hf_data = hf_admissions_sorted.copy()

hf_data = hf_data.merge(
    patients[['subject_id','gender', 'dob']],
    on='subject_id',
    how='left',
    suffixes=('', '_patient')
)

# Apply to dataframe
hf_data['age'] = hf_data.apply(
    lambda row: calculate_age_robust(row['dob'], row['admittime']), 
    axis=1
)

# Diagnostic output
print("Age calculation results:")
print(f"Valid ages: {hf_data['age'].notna().sum()}")
print(f"NaN ages: {hf_data['age'].isna().sum()}")
print("Top reasons for NaN ages:")

nan_reasons = hf_data[hf_data['age'].isna()]
print(f"1. Missing dob: {nan_reasons['dob'].isna().sum()}")
print(f"2. Missing admittime: {nan_reasons['admittime'].isna().sum()}")
print(f"3. Invalid date ranges: {((nan_reasons['dob'].notna()) & (nan_reasons['admittime'].notna())).sum()}")

# Show distribution of calculated ages
print("\nAge distribution:")
print(hf_data['age'].describe())

# Verify with readmitted patients
if 'readmit_30' in hf_data.columns:
    print("\nAge distribution for readmitted patients:")
    print(hf_data[hf_data['readmit_30'] == 1]['age'].describe())

Age calculation results:
Valid ages: 12697
NaN ages: 1343
Top reasons for NaN ages:
1. Missing dob: 0
2. Missing admittime: 0
3. Invalid date ranges: 1343

Age distribution:
count    12697.000000
mean        69.758132
std         13.133650
min          0.000000
25%         62.000000
50%         72.000000
75%         80.000000
max        120.000000
Name: age, dtype: float64

Age distribution for readmitted patients:
count    875.000000
mean      69.241143
std       13.359838
min       20.000000
25%       61.000000
50%       72.000000
75%       80.000000
max      120.000000
Name: age, dtype: float64


In [8]:
hf_data[hf_data['readmit_30'] == 1]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag,next_admittime,days_to_readmit,readmit_30,gender,dob,age
14,68,170467,2173-12-15 16:16:00,2174-01-03 18:30:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2174-01-04 22:21:00,1.160417,1,F,2132-02-29 00:00:00.000,41.0
99,356,135591,2132-08-05 00:09:00,2132-08-13 17:41:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2132-08-31 17:05:00,17.975000,1,M,2073-10-07 00:00:00.000,58.0
150,518,143946,2109-06-27 04:45:00,2109-06-29 17:00:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-07 07:49:00,7.617361,1,M,2062-09-18 00:00:00.000,46.0
151,518,153168,2109-07-07 07:49:00,2109-07-09 12:16:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-17 04:21:00,7.670139,1,M,2062-09-18 00:00:00.000,46.0
154,530,151499,2119-03-13 16:33:00,2119-04-04 12:00:00,NaT,EMERGENCY,REHAB/DISTINCT PART HOSP,MARRIED,0,2119-04-07 16:14:00,3.176389,1,M,2039-10-16 00:00:00.000,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13988,99439,143661,2179-02-22 01:26:00,2179-02-25 15:50:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-18 11:27:00,20.817361,1,M,2095-01-21 00:00:00.000,84.0
13989,99439,145935,2179-03-18 11:27:00,2179-03-20 18:00:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-27 20:45:00,7.114583,1,M,2095-01-21 00:00:00.000,84.0
14005,99538,194801,2180-02-13 17:46:00,2180-02-26 15:35:00,NaT,EMERGENCY,SNF,UNKNOWN (DEFAULT),0,2180-03-12 16:01:00,15.018056,1,F,2107-11-11 00:00:00.000,72.0
14011,99562,129689,2183-02-02 15:13:00,2183-02-18 13:28:00,NaT,URGENT,SNF,MARRIED,0,2183-03-06 01:31:00,15.502083,1,M,2120-12-29 00:00:00.000,62.0


In [9]:
hf_data.shape

(14040, 15)

In [10]:
print('diagnoses_icd:', diagnoses_icd.shape)
print(diagnoses_icd.columns)
diagnoses_icd

diagnoses_icd: (651047, 5)
Index(['row_id', 'subject_id', 'hadm_id', 'seq_num', 'icd9_code'], dtype='object')


,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [11]:
# Step 1: Clean ICD-9 codes (convert all to strings)
diagnoses_icd['icd9_code'] = (
    diagnoses_icd['icd9_code']
    .astype(str)  # Convert all to strings
    .str.replace(r'\.0$', '', regex=True)  # Remove trailing .0 from floats
)

# Step 2: Safe aggregation (handles empty groups)
diagnoses_agg = (
    diagnoses_icd.groupby(['subject_id', 'hadm_id'])['icd9_code']
    .apply(lambda codes: ','.join(codes.dropna()))  # Skip NA values
    .reset_index(name='all_diagnoses')
)

# Step 3: Count diagnoses per admission
diagnoses_count = (
    diagnoses_icd.groupby(['subject_id', 'hadm_id'])['icd9_code']
    .count()
    .reset_index(name='diagnosis_count')
)

# Step 4: Merge with original data
hf_data = hf_data.merge(
    diagnoses_agg,
    on=['subject_id', 'hadm_id'],
    how='left'
).merge(
    diagnoses_count,
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Fill NA values
hf_data['diagnosis_count'] = hf_data['diagnosis_count'].fillna(0)
hf_data['all_diagnoses'] = hf_data['all_diagnoses'].fillna('No_Diagnoses')

# Verify
print(f"Final columns: {hf_data.columns.tolist()}")
print(f"Diagnoses coverage: {hf_data['all_diagnoses'].ne('No_Diagnoses').mean():.1%}")

Final columns: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'discharge_location', 'marital_status', 'hospital_expire_flag', 'next_admittime', 'days_to_readmit', 'readmit_30', 'gender', 'dob', 'age', 'all_diagnoses', 'diagnosis_count']
Diagnoses coverage: 100.0%


In [12]:
hf_data[hf_data['readmit_30'] == 1]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag,next_admittime,days_to_readmit,readmit_30,gender,dob,age,all_diagnoses,diagnosis_count
14,68,170467,2173-12-15 16:16:00,2174-01-03 18:30:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2174-01-04 22:21:00,1.160417,1,F,2132-02-29 00:00:00.000,41.0,"042,486,4254,42820,4280,5849,5859,2639,2848,27...",13
99,356,135591,2132-08-05 00:09:00,2132-08-13 17:41:00,NaT,EMERGENCY,HOME HEALTH CARE,SINGLE,0,2132-08-31 17:05:00,17.975000,1,M,2073-10-07 00:00:00.000,58.0,"80504,4280,4254,2768,42731,5939,V4502,V433,E8859",9
150,518,143946,2109-06-27 04:45:00,2109-06-29 17:00:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-07 07:49:00,7.617361,1,M,2062-09-18 00:00:00.000,46.0,"042,40391,4280,25061,5855,78701,3572,V1581,333...",10
151,518,153168,2109-07-07 07:49:00,2109-07-09 12:16:00,NaT,EMERGENCY,HOME,MARRIED,0,2109-07-17 04:21:00,7.670139,1,M,2062-09-18 00:00:00.000,46.0,"042,25041,40391,4280,41519,5856,25061,5363,583...",10
154,530,151499,2119-03-13 16:33:00,2119-04-04 12:00:00,NaT,EMERGENCY,REHAB/DISTINCT PART HOSP,MARRIED,0,2119-04-07 16:14:00,3.176389,1,M,2039-10-16 00:00:00.000,79.0,"3962,4160,9971,42731,42820,4280,41401,41402,44...",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13988,99439,143661,2179-02-22 01:26:00,2179-02-25 15:50:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-18 11:27:00,20.817361,1,M,2095-01-21 00:00:00.000,84.0,"34830,42842,5849,4168,43883,496,32723,4019,424...",23
13989,99439,145935,2179-03-18 11:27:00,2179-03-20 18:00:00,NaT,EMERGENCY,SNF,MARRIED,0,2179-03-27 20:45:00,7.114583,1,M,2095-01-21 00:00:00.000,84.0,"42833,5990,5849,5854,42732,7802,496,32723,4388...",18
14005,99538,194801,2180-02-13 17:46:00,2180-02-26 15:35:00,NaT,EMERGENCY,SNF,UNKNOWN (DEFAULT),0,2180-03-12 16:01:00,15.018056,1,F,2107-11-11 00:00:00.000,72.0,"41071,42833,5849,5119,99859,9971,9980,5990,401...",28
14011,99562,129689,2183-02-02 15:13:00,2183-02-18 13:28:00,NaT,URGENT,SNF,MARRIED,0,2183-03-06 01:31:00,15.502083,1,M,2120-12-29 00:00:00.000,62.0,"99659,53240,5762,42833,2851,7907,00845,5849,58...",26


In [13]:
# Analyze diagnoses distribution
diagnoses_detail = diagnoses_icd.merge(
    hf_data[['subject_id', 'hadm_id', 'readmit_30']], 
    on=['subject_id', 'hadm_id'],
    how='inner'
)

# Get most common diagnoses
print("\nMost common diagnoses:")
print(diagnoses_detail['icd9_code'].value_counts().head(10))

# Compare diagnoses between readmitted and non-readmitted patients
print("\nTop diagnoses for readmitted patients:")
readmit_diagnoses = diagnoses_detail[diagnoses_detail['readmit_30']==1]['icd9_code'].value_counts().head(10)
print(readmit_diagnoses)

print("\nTop diagnoses for non-readmitted patients:")
non_readmit_diagnoses = diagnoses_detail[diagnoses_detail['readmit_30']==0]['icd9_code'].value_counts().head(10)
print(non_readmit_diagnoses)


Most common diagnoses:
icd9_code
4280     13111
42731     6251
4019      5115
41401     4907
5849      4083
25000     3410
51881     3010
2724      2797
5990      2469
496       2059
Name: count, dtype: int64

Top diagnoses for readmitted patients:
icd9_code
4280     887
42731    436
5849     327
41401    299
4019     297
25000    248
51881    203
2724     194
5990     192
496      157
Name: count, dtype: int64

Top diagnoses for non-readmitted patients:
icd9_code
4280     12224
42731     5815
4019      4818
41401     4608
5849      3756
25000     3162
51881     2807
2724      2603
5990      2277
496       1902
Name: count, dtype: int64


In [14]:
checkpoint = hf_data

In [38]:
hf_data = checkpoint.copy()

In [39]:
# First merge lab_events with d_labitems to get the test names
lab_events_with_names = pd.merge(
    labevents,
    d_labitems[['itemid', 'label', 'category']],
    on='itemid',
    how='inner'  # Changed from 'left' to ensure only valid lab tests
)

# Filter lab events for abnormal results
abnormal_lab_events = lab_events_with_names[
    lab_events_with_names['flag'].str.upper().isin(['ABNORMAL','HIGH','LOW']) & 
    lab_events_with_names['flag'].notna()  # Ensure flag is not NaN
]

# Now merge with hf_data
hf_lab_events = pd.merge(
    hf_data[['subject_id', 'hadm_id']],
    abnormal_lab_events,
    on=['subject_id', 'hadm_id'],
    how='inner'  # Changed from 'left' to only keep matches
)

# Verify data before creating summary
print("Number of lab events:", len(hf_lab_events))
print("Sample of lab events:")
print(hf_lab_events[['itemid', 'label', 'flag']].head())

# Now group by hadm_id and create structured data
def create_lab_summary(group):
    valid_tests = group[group['itemid'].notna()]  # Only include valid tests
    return {
        'lab_tests': valid_tests[['itemid', 'label', 'charttime', 'valuenum', 'valueuom', 'flag']].to_dict('records'),
        'test_count': len(valid_tests),
        'unique_tests': valid_tests['label'].nunique()
    }

hf_lab_summary = hf_lab_events.groupby('hadm_id').apply(create_lab_summary).reset_index(name='lab_data')

# Finally merge this back with the original hf_data
final_hf_data = pd.merge(
    hf_data,
    hf_lab_summary,
    on='hadm_id',
    how='left'
)

# Verify the results
print("\nSample of first lab_data entry:")
print(final_hf_data['lab_data'].iloc[0])

Number of lab events: 2742754
Sample of lab events:
   itemid           label      flag
0   50893  Calcium, Total  abnormal
1   50902        Chloride  abnormal
2   50912      Creatinine  abnormal
3   50970       Phosphate  abnormal
4   50971       Potassium  abnormal

Sample of first lab_data entry:
{'lab_tests': [{'itemid': 50893, 'label': 'Calcium, Total', 'charttime': '2101-10-20 16:40:00.000', 'valuenum': 8.2, 'valueuom': 'mg/dL', 'flag': 'abnormal'}, {'itemid': 50902, 'label': 'Chloride', 'charttime': '2101-10-20 16:40:00.000', 'valuenum': 99.0, 'valueuom': 'mEq/L', 'flag': 'abnormal'}, {'itemid': 50912, 'label': 'Creatinine', 'charttime': '2101-10-20 16:40:00.000', 'valuenum': 3.2, 'valueuom': 'mg/dL', 'flag': 'abnormal'}, {'itemid': 50970, 'label': 'Phosphate', 'charttime': '2101-10-20 16:40:00.000', 'valuenum': 4.8, 'valueuom': 'mg/dL', 'flag': 'abnormal'}, {'itemid': 50971, 'label': 'Potassium', 'charttime': '2101-10-20 16:40:00.000', 'valuenum': 5.4, 'valueuom': 'mEq/L', 'fla

C:\Users\adity\AppData\Local\Temp\ipykernel_22316\224808937.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hf_lab_summary = hf_lab_events.groupby('hadm_id').apply(create_lab_summary).reset_index(name='lab_data')


In [40]:
final_hf_data.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'discharge_location', 'marital_status',
       'hospital_expire_flag', 'next_admittime', 'days_to_readmit',
       'readmit_30', 'gender', 'dob', 'age', 'all_diagnoses',
       'diagnosis_count', 'lab_data'],
      dtype='object')

In [41]:

final_hf_data['lab_data'] = final_hf_data['lab_data'].apply(lambda x: x if isinstance(x, dict) else {})

# Check the structure of the lab data
print("Lab data structure:")
final_hf_data[['hadm_id', 'lab_data']]

Lab data structure:


,hadm_id,lab_data
0,145834,"{'lab_tests': [{'itemid': 50893, 'label': 'Cal..."
1,150750,"{'lab_tests': [{'itemid': 50821, 'label': 'pO2..."
2,109451,"{'lab_tests': [{'itemid': 51274, 'label': 'PT'..."
3,197661,"{'lab_tests': [{'itemid': 51248, 'label': 'MCH..."
4,104557,"{'lab_tests': [{'itemid': 50811, 'label': 'Hem..."
...,...,...
14035,151454,"{'lab_tests': [{'itemid': 50912, 'label': 'Cre..."
14036,112748,"{'lab_tests': [{'itemid': 51006, 'label': 'Ure..."
14037,183791,"{'lab_tests': [{'itemid': 51265, 'label': 'Pla..."
14038,151118,"{'lab_tests': [{'itemid': 50821, 'label': 'pO2..."


In [42]:
lab_checkpoint = final_hf_data

In [49]:
final_hf_data[final_hf_data['hospital_expire_flag'] == 1]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,discharge_location,marital_status,hospital_expire_flag,next_admittime,days_to_readmit,readmit_30,gender,dob,age,all_diagnoses,diagnosis_count,lab_data
1,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,DEAD/EXPIRED,NaN,1,NaT,NaN,0,M,2108-01-26 00:00:00.000,41.0,"431,5070,4280,5849,2765,4019",6,"{'lab_tests': [{'itemid': 50821, 'label': 'pO2..."
12,56,181711,2104-01-02 02:01:00,2104-01-08 10:30:00,2104-01-08 10:30:00,EMERGENCY,DEAD/EXPIRED,NaN,1,NaT,NaN,0,F,1804-01-02 00:00:00.000,NaN,"1983,1628,53140,2761,4280,78039,2720,4019",8,"{'lab_tests': [{'itemid': 50902, 'label': 'Chl..."
20,111,155897,2144-07-01 04:12:00,2144-07-01 14:55:00,2144-07-01 14:55:00,EMERGENCY,DEAD/EXPIRED,MARRIED,1,NaT,NaN,0,F,2075-07-16 00:00:00.000,68.0,"0389,5849,486,51881,78552,2866,496,4019,4280,2...",12,"{'lab_tests': [{'itemid': 50868, 'label': 'Ani..."
23,124,138376,2165-12-31 18:55:00,2166-02-01 06:55:00,2166-02-01 06:55:00,EMERGENCY,DEAD/EXPIRED,MARRIED,1,NaT,NaN,0,M,2090-11-19 00:00:00.000,75.0,"56983,56089,49121,99859,0389,99592,99831,486,5...",19,"{'lab_tests': [{'itemid': 51249, 'label': 'MCH..."
36,164,182743,2116-12-28 15:32:00,2117-01-16 03:20:00,2117-01-16 03:20:00,EMERGENCY,DEAD/EXPIRED,WIDOWED,1,NaT,NaN,0,M,2034-04-26 00:00:00.000,82.0,"51884,4280,515,4275,486,2554,42731,99594,4538,...",11,"{'lab_tests': [{'itemid': 50931, 'label': 'Glu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13967,99115,132152,2193-12-25 19:00:00,2194-01-14 00:01:00,2194-01-14 00:01:00,EMERGENCY,DEAD/EXPIRED,SINGLE,1,NaT,NaN,0,M,2142-08-26 00:00:00.000,51.0,"42731,5856,51881,570,34982,42823,0389,99592,99...",34,"{'lab_tests': [{'itemid': 50878, 'label': 'Asp..."
13997,99469,179324,2183-04-16 23:20:00,2183-05-13 16:20:00,2183-05-13 16:20:00,EMERGENCY,DEAD/EXPIRED,MARRIED,1,NaT,NaN,0,M,2115-02-28 00:00:00.000,68.0,"03812,78552,48242,51881,5070,42823,5845,5180,4...",39,"{'lab_tests': [{'itemid': 50912, 'label': 'Cre..."
13998,99472,161362,2193-12-22 19:02:00,2194-01-01 14:10:00,2194-01-01 14:10:00,EMERGENCY,DEAD/EXPIRED,WIDOWED,1,NaT,NaN,0,F,2111-08-06 00:00:00.000,82.0,"0389,41071,42821,51881,570,5070,5845,486,78552...",20,"{'lab_tests': [{'itemid': 50983, 'label': 'Sod..."
14001,99491,125502,2139-01-21 16:32:00,2139-02-17 21:30:00,2139-02-17 21:30:00,EMERGENCY,DEAD/EXPIRED,MARRIED,1,NaT,NaN,0,M,2055-08-06 00:00:00.000,83.0,"51881,5570,42823,5845,78001,570,0389,99592,997...",39,"{'lab_tests': [{'itemid': 50820, 'label': 'pH'..."


In [50]:
final_hf_data.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'discharge_location', 'marital_status',
       'hospital_expire_flag', 'next_admittime', 'days_to_readmit',
       'readmit_30', 'gender', 'dob', 'age', 'all_diagnoses',
       'diagnosis_count', 'lab_data'],
      dtype='object')

In [51]:
final_checkpoint = final_hf_data.copy()

In [59]:
dataset = final_hf_data.drop(['subject_id', 'hadm_id' , 'admittime', 'dischtime', 'deathtime', 'next_admittime','dob'], axis=1)

In [60]:
dataset.to_csv('modified_dataset.csv', index=False)

In [68]:
dataset

,admission_type,discharge_location,marital_status,hospital_expire_flag,days_to_readmit,readmit_30,gender,age,all_diagnoses,diagnosis_count,lab_data
0,EMERGENCY,SNF,MARRIED,0,NaN,0,M,76.0,"0389,78559,5849,4275,41071,4280,6826,4254,2639",9,"{'lab_tests': [{'itemid': 50893, 'label': 'Cal..."
1,EMERGENCY,DEAD/EXPIRED,NaN,1,NaN,0,M,41.0,"431,5070,4280,5849,2765,4019",6,"{'lab_tests': [{'itemid': 50821, 'label': 'pO2..."
2,EMERGENCY,REHAB/DISTINCT PART HOSP,MARRIED,0,NaN,0,M,87.0,"41071,78551,5781,5849,40391,4280,4592,5070,427...",18,"{'lab_tests': [{'itemid': 51274, 'label': 'PT'..."
3,EMERGENCY,HOME,SINGLE,0,NaN,0,M,72.0,"99604,4271,4280,42731,41401,412,5939,2720,60000",9,"{'lab_tests': [{'itemid': 51248, 'label': 'MCH..."
4,URGENT,HOME HEALTH CARE,MARRIED,0,NaN,0,M,NaN,"99674,4160,4111,4281,4271,41401,4019,2720,5939",9,"{'lab_tests': [{'itemid': 50811, 'label': 'Hem..."
...,...,...,...,...,...,...,...,...,...,...,...
14035,EMERGENCY,HOME HEALTH CARE,MARRIED,0,28.154167,1,M,65.0,"42823,4254,2875,42731,3970,5303,4280,V5861,45829",9,"{'lab_tests': [{'itemid': 50912, 'label': 'Cre..."
14036,EMERGENCY,HOME,MARRIED,0,35.188194,0,M,65.0,"4280,42823,5849,4254,2763,42731,78729,53081,V4...",13,"{'lab_tests': [{'itemid': 51006, 'label': 'Ure..."
14037,EMERGENCY,SHORT TERM HOSPITAL,MARRIED,0,NaN,0,M,65.0,"5849,42731,4280,2875,59689,53081,56400,78720,V...",16,"{'lab_tests': [{'itemid': 51265, 'label': 'Pla..."
14038,ELECTIVE,HOME,MARRIED,0,NaN,0,M,47.0,"56211,0389,5570,5849,99592,56081,78959,5538,78...",17,"{'lab_tests': [{'itemid': 50821, 'label': 'pO2..."


In [69]:
dataset.columns

Index(['admission_type', 'discharge_location', 'marital_status',
       'hospital_expire_flag', 'days_to_readmit', 'readmit_30', 'gender',
       'age', 'all_diagnoses', 'diagnosis_count', 'lab_data'],
      dtype='object')

In [67]:
dataset['lab_data'].iloc[0]

{'lab_tests': [{'itemid': 50893,
   'label': 'Calcium, Total',
   'charttime': '2101-10-20 16:40:00.000',
   'valuenum': 8.2,
   'valueuom': 'mg/dL',
   'flag': 'abnormal'},
  {'itemid': 50902,
   'label': 'Chloride',
   'charttime': '2101-10-20 16:40:00.000',
   'valuenum': 99.0,
   'valueuom': 'mEq/L',
   'flag': 'abnormal'},
  {'itemid': 50912,
   'label': 'Creatinine',
   'charttime': '2101-10-20 16:40:00.000',
   'valuenum': 3.2,
   'valueuom': 'mg/dL',
   'flag': 'abnormal'},
  {'itemid': 50970,
   'label': 'Phosphate',
   'charttime': '2101-10-20 16:40:00.000',
   'valuenum': 4.8,
   'valueuom': 'mg/dL',
   'flag': 'abnormal'},
  {'itemid': 50971,
   'label': 'Potassium',
   'charttime': '2101-10-20 16:40:00.000',
   'valuenum': 5.4,
   'valueuom': 'mEq/L',
   'flag': 'abnormal'},
  {'itemid': 50878,
   'label': 'Asparate Aminotransferase (AST)',
   'charttime': '2101-10-22 04:00:00.000',
   'valuenum': 67.0,
   'valueuom': 'IU/L',
   'flag': 'abnormal'},
  {'itemid': 50882,
   